In [2]:
from keras.models import load_model
import cv2
import numpy as np
import mediapipe as mp

In [3]:
model = load_model('action.h5')

In [4]:
import os

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

def generate_face(results):

    if not results.face_landmarks:
        return np.zeros(160)

    face = np.array([[res.x, res.y] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros(468*3)
    x_min, x_max, y_min, y_max = np.min(face[:,0]), np.max(face[:,0]), np.min(face[:,1]), np.max(face[:,1])
    closest_points = []

    for y in np.linspace(y_min,y_max,20):
        closest_points.append(np.argmin(np.abs(face[:,0] - x_min) + np.abs(face[:,1] - y)))
        closest_points.append(np.argmin(np.abs(face[:,0] - x_max) + np.abs(face[:,1] - y)))

    for x in np.linspace(x_min,x_max,20):
        closest_points.append(np.argmin(np.abs(face[:,0] - x) + np.abs(face[:,1] - y_max)))
        closest_points.append(np.argmin(np.abs(face[:,0] - x) + np.abs(face[:,1] - y_min)))

    return np.array([[face[i,0], face[i,1]] for i in closest_points]).flatten()


def extract_keypoints(results):
    lh = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*2)
    rh = np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*2)
    return np.concatenate([generate_face(results), lh, rh])

colors = [(245,117,16), (117,245,16), (16,117,245), (255,117,245), (255,117,245)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(output_frame, f"{round(prob*100, 2)}%", (120, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

    return output_frame

# create a list of all subfolders
actions = []
for file in os.listdir("MP_Data"):
    if os.path.isdir(os.path.join("MP_Data", file)):
        actions.append(file)

actions = np.array(actions)

label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'world': 1, 'from': 2, 'germany': 3, 'none': 4}

In [5]:
no_sequences = 200
sequence_length = 20
DATA_PATH = os.path.join('MP_Data')

In [6]:
from IPython.core.display_functions import clear_output

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.8, min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)

        sequence = sequence[-sequence_length:]

        clear_output(wait=True)

        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(sum(res))
            if np.max(res) > threshold:
                print(actions[np.argmax(res)])

            image = prob_viz(res, actions, image, colors)


        cv2.imshow('OpenCV Feed', image)


        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 13ms/step
0.9999999602605385
none


In [7]:
res = model.predict(np.expand_dims(sequence, axis=0))[0]
actions[np.argmax(res)]



1/1 [==============================] - 0s 13ms/step


'none'